# Businesses - Brand and Competition Locations

The geography level methods and enrichment methods detailed in the previous notebooks enable retrieving location and demographic information describing where and who customers are. Next though, we need to retrieve information about the retail landscape surrounding these potential customers. Knowing the locations of brand and competitor locations is essential to modeling the effects of competition and cannibalization.

In [1]:
from pathlib import Path
import sys

from dm import Country

# load the "autoreload" extension so as src code is changed, the changes are picked up in the dataframe
%load_ext autoreload
%autoreload 2

project_parent = Path('./').absolute().parent
dir_data = project_parent/'data'
dir_int = dir_data/'interim'
gdb_int = dir_int/'interim.gdb'

Getting started is not dramactically different than before - instantiate a country object intsance and delineate an area of interest.

In [2]:
usa = Country('USA')
aoi_df = usa.cbsas.get('seattle')

aoi_mp = aoi_df.spatial.plot()
aoi_mp.basemap = 'gray-vector'
aoi_mp

MapView(layout=Layout(height='400px', width='100%'))

## Brand Locations

Then, business locations can be retrieved using the business name.

In [3]:
biz_df = usa.business.get_by_name('Ace Hardware', aoi_df)
biz_df.head()

,OBJECTID,LOCNUM,CONAME,STREET,CITY,STATE,STATE_NAME,ZIP,ZIP4,NAICS,...,SQFTCODE,LOC_NAME,STATUS,SCORE,SOURCE,REC_TYPE,SHAPE,id,brand_name,brand_name_category
0,894611,216082099,SOUTH END ACE HARDWARE,PACIFIC AVE S,SPANAWAY,WA,Washington,98387,8395,44413005,...,4,PointAddress,M,100.000000,INFOGROUP,0,"{""x"": -122.43443850000003, ""y"": 47.08713600000...",216082099,SOUTH END ACE HARDWARE,SOUTH END ACE HARDWARE
1,1632160,371889957,GRAHAM ACE HARDWARE,224TH ST E,GRAHAM,WA,Washington,98338,5704,44413005,...,5,PointAddress,M,100.000000,INFOGROUP,0,"{""x"": -122.29924949999995, ""y"": 47.05465500000...",371889957,GRAHAM ACE HARDWARE,GRAHAM ACE HARDWARE
2,5379592,460556608,OAKBROOK ACE HARDWARE,STEILACOOM BLVD SW,LAKEWOOD,WA,Washington,98498,6154,44413005,...,3,PointAddress,M,100.000000,INFOGROUP,0,"{""x"": -122.54611950000002, ""y"": 47.18077199999...",460556608,OAKBROOK ACE HARDWARE,OAKBROOK ACE HARDWARE
3,2624850,405129289,GIG HARBOR ACE HARDWARE,POINT FOSDICK DR,GIG HARBOR,WA,Washington,98335,1711,44413005,...,6,PointAddress,M,99.546875,INFOGROUP,0,"{""x"": -122.58131400000003, ""y"": 47.30224499999...",405129289,GIG HARBOR ACE HARDWARE,GIG HARBOR ACE HARDWARE
4,2484547,404324160,AGRISHOP ACE HARDWARE,S 12TH ST,TACOMA,WA,Washington,98405,3004,44413005,...,5,PointAddress,M,100.000000,INFOGROUP,0,"{""x"": -122.463747, ""y"": 47.24991900000006, ""sp...",404324160,AGRISHOP ACE HARDWARE,AGRISHOP ACE HARDWARE


In [4]:
biz_mp = biz_df.spatial.plot()
biz_mp.basemap = 'gray-vector'
biz_mp

MapView(layout=Layout(height='400px', width='100%'))

Businesses can also be looked up by their category code. By default, this uses NAICS, but SIC can also be used. Also, something very useful for chains, especially QSR's, is to identify local brands collectively. This is calculated based on the company name, how many times the company name appears. In this case, the method does not work as well since every Ace Hardware has a unique name based on the location.

In [5]:
naics_df = usa.business.get_by_code(44413005, aoi_df, local_threshold=2)

print(f'rows retrieved: {len(naics_df.index)}\n')

naics_df.head()

rows retrieved: 112



,OBJECTID,LOCNUM,CONAME,STREET,CITY,STATE,STATE_NAME,ZIP,ZIP4,NAICS,...,SQFTCODE,LOC_NAME,STATUS,SCORE,SOURCE,REC_TYPE,SHAPE,id,brand_name,brand_name_category
0,894611,216082099,SOUTH END ACE HARDWARE,PACIFIC AVE S,SPANAWAY,WA,Washington,98387,8395,44413005,...,4,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -122.43443850000003, ""y"": 47.08713600000...",216082099,SOUTH END ACE HARDWARE,local_brand
1,1632160,371889957,GRAHAM ACE HARDWARE,224TH ST E,GRAHAM,WA,Washington,98338,5704,44413005,...,5,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -122.29924949999995, ""y"": 47.05465500000...",371889957,GRAHAM ACE HARDWARE,local_brand
2,1391610,303665954,CAPITOL LUMBER INC,KEY PENINSULA HWY NW,LAKEBAY,WA,Washington,98349,9327,44413005,...,3,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -122.744898, ""y"": 47.340161999999985, ""s...",303665954,CAPITOL LUMBER INC,local_brand
3,11220705,740405810,CASCADE WHOLESALE HARDWARE,PACIFIC HWY SW,LAKEWOOD,WA,Washington,98499,1027,44413005,...,3,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -122.51642849999998, ""y"": 47.142639, ""sp...",740405810,CASCADE WHOLESALE HARDWARE,local_brand
4,751596,175096239,LAKEWOOD HARDWARE & PAINT,LAKEWOOD DR SW,LAKEWOOD,WA,Washington,98499,3966,44413005,...,5,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -122.50837799999996, ""y"": 47.17514700000...",175096239,LAKEWOOD HARDWARE & PAINT,local_brand


Further, it frequently is also useful to retrieve broader categories by simply truncating two digits at a time to be less specific.

In [6]:
naics_big_df = usa.business.get_by_code(4441, aoi_df, local_threshold=2)

print(f'rows retrieved: {len(naics_big_df.index):,}\n')

naics_big_df.head()

rows retrieved: 1,565



,OBJECTID,LOCNUM,CONAME,STREET,CITY,STATE,STATE_NAME,ZIP,ZIP4,NAICS,...,SQFTCODE,LOC_NAME,STATUS,SCORE,SOURCE,REC_TYPE,SHAPE,id,brand_name,brand_name_category
0,12744705,973622731,ARROW LUMBER & HARDWARE,CENTER ST E,EATONVILLE,WA,Washington,98328,7449,44419044,...,5,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -122.2617105, ""y"": 46.86600600000002, ""s...",973622731,ARROW LUMBER & HARDWARE,ARROW LUMBER & HARDWARE
1,912883,218001303,TIN MEN SUPPLY INC,STATE ROUTE 507 S,ROY,WA,Washington,98580,,44419055,...,3,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -122.55250049999998, ""y"": 46.94270400000...",218001303,TIN MEN SUPPLY INC,local_brand
2,5549815,481554129,MILES SAND & GRAVEL CO,STATE ROUTE 507 S,ROY,WA,Washington,98580,8745,44419012,...,5,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -122.5563975, ""y"": 46.972286999999994, ""...",481554129,MILES SAND & GRAVEL CO,local_brand
3,12660147,961135167,MC KENNA DOOR,46TH AVE S,ROY,WA,Washington,98580,7623,44419017,...,2,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -122.49186299999998, ""y"": 46.93944600000...",961135167,MC KENNA DOOR,local_brand
4,751354,175068543,INSTANT BLEND CONCRETE,52ND AVENUE CT E,SPANAWAY,WA,Washington,98387,5798,44419012,...,2,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -122.36186249999994, ""y"": 47.00474999999...",175068543,INSTANT BLEND CONCRETE,local_brand


## Competitor Locations

Also, competitors can easily be located using the previously retrieved data, or using a brand name to exclude from retrieved results. The latter is extremely useful when using locations the organization already has for the brand locations, but needing competitor locations for modeling.

In [7]:
# comp_df = usa.business.get_competition(biz_df, aoi_df)
comp_df = usa.business.get_competition('Ace Hardware', aoi_df, local_threshold=2)

comp_df.head()

,OBJECTID,LOCNUM,CONAME,STREET,CITY,STATE,STATE_NAME,ZIP,ZIP4,NAICS,...,SQFTCODE,LOC_NAME,STATUS,SCORE,SOURCE,REC_TYPE,SHAPE,id,brand_name,brand_name_category
0,1391610,303665954,CAPITOL LUMBER INC,KEY PENINSULA HWY NW,LAKEBAY,WA,Washington,98349,9327,44413005,...,3,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -122.744898, ""y"": 47.340161999999985, ""s...",303665954,CAPITOL LUMBER INC,local_brand
1,11220705,740405810,CASCADE WHOLESALE HARDWARE,PACIFIC HWY SW,LAKEWOOD,WA,Washington,98499,1027,44413005,...,3,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -122.51642849999998, ""y"": 47.142639, ""sp...",740405810,CASCADE WHOLESALE HARDWARE,local_brand
2,751596,175096239,LAKEWOOD HARDWARE & PAINT,LAKEWOOD DR SW,LAKEWOOD,WA,Washington,98499,3966,44413005,...,5,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -122.50837799999996, ""y"": 47.17514700000...",175096239,LAKEWOOD HARDWARE & PAINT,local_brand
3,8587816,716987694,WILCO FARM STORES-GIG HARBOR,E HIGHWAY 20,GIG HARBOR,WA,Washington,98335,,44413005,...,3,Postal,M,85.0,INFOGROUP,0,"{""x"": -122.64751459999998, ""y"": 47.29498229999...",716987694,WILCO FARM STORES-GIG HARBOR,local_brand
4,3813730,425066759,MCLENDON HARDWARE INC,N PEARL ST,TACOMA,WA,Washington,98406,7921,44413005,...,7,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -122.51411550000003, ""y"": 47.25932400000...",425066759,MCLENDON HARDWARE INC,MCLENDON HARDWARE INC


Also, just as before, the returned results are Spatially Enabled DataFrames, so they can be easily saved for follow on analysis or visualization in ArcGIS Pro.

In [8]:
biz_df.spatial.to_featureclass(gdb_int/'loc_brand')

'D:\\projects\\demographic-modeling-module\\data\\interim\\interim.gdb\\loc_brand'

In [9]:
comp_df.spatial.to_featureclass(gdb_int/'loc_comp')

'D:\\projects\\demographic-modeling-module\\data\\interim\\interim.gdb\\loc_comp'